In [1]:
def Our_model():
    from keras.layers import Input, Embedding, Flatten, Dot, Dense, Dropout
    from keras.models import Model
    # For each sample we input the integer identifiers
    # of a single user and a single item
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')

    embedding_size = 30
    user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                               input_length=1, name='user_embedding')(user_id_input)

    item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                               input_length=1, name='item_embedding')(item_id_input)

    # reshape from shape: (batch_size, input_length, embedding_size)
    # to shape: (batch_size, input_length * embedding_size) which is
    # equal to shape: (batch_size, embedding_size)
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)

    y = Dot(axes=1)([user_vecs, item_vecs])
    x = Dense(64, activation='relu')(y)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    y = Dense(1)(x)
    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    model.compile(optimizer="adam", loss="MAE")
    return model

In [2]:

import requests
from time import sleep 
import numpy as np
from keras.callbacks import EarlyStopping
user_id= '0SIMZCTG7KZACJPQB7FU'
base_url='http://52.47.62.31'
url_reset=base_url+'/reset'
url_predict=base_url+'/predict'
params={'user_id':user_id}
data= requests.get(url_reset, params=params).json()
nb_users=data['nb_users']
nb_items=data['nb_items']
rating_history=data['rating_history']
item_history=data['item_history']
next_user=data['next_user']
next_item=data['next_item']
#prediction=3
#params['predicted_score']=prediction
nb_samples=1000
mse,mae=0,0
user_history=data['user_history']
model=Our_model()
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
history = model.fit([user_history, item_history], rating_history,
                    batch_size=64, epochs=20, validation_split=0.1,
                    shuffle=True, callbacks=[early_stopping])
for i in range(nb_samples):
    sleep(0.05)
    prediction = model.predict([np.array([next_user]), np.array([next_item])] )
    params['predicted_score']=prediction
    d=requests.get(url_predict, params=params).json()
    rating=d['rating']
    #print(f'user:{next_user},item:{next_item},rating:{rating},prediction:{prediction}')
    next_user=d['next_user']
    next_item=d['next_item']
    mse+=(rating-prediction)**2
    mae+=abs(rating-prediction)
print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

C:\Users\RAMZI\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 1s 61us/step - loss: 1.7595 - val_loss: 1.1422
Epoch 2/20
9000/9000 [==============================] - 0s 23us/step - loss: 1.1532 - val_loss: 0.8334
Epoch 3/20
9000/9000 [==============================] - 0s 23us/step - loss: 0.6850 - val_loss: 0.5320
Epoch 4/20
9000/9000 [==============================] - 0s 23us/step - loss: 0.5139 - val_loss: 0.4646
Epoch 5/20
9000/9000 [==============================] - 0s 28us/step - loss: 0.4164 - val_loss: 0.4241
Epoch 6/20
9000/9000 [==============================] - 0s 27us/step - loss: 0.3734 - val_loss: 0.4077
Epoch 7/20
9000/9000 [==============================] - 0s 21us/step - loss: 0.3492 - val_loss: 0.3935
Epoch 8/20
9000/9000 [==============================] - 0s 21us/step - loss: 0.3309 - val_loss: 0.3906
Epoch 9/20
9000/9000 [==============================] - 0s 23us/step - loss: 0.3109 - val_loss: 0.3645
Epoch 10/20
9000/9000 [==